# Weather-Aware MMM: Comprehensive Model Validation

**Purpose**: Reproducible validation of weather-aware MMM models against objective performance thresholds.

**Date**: 2025-10-23

**Model Requirements**:
- R² ≥ 0.50 (minimum acceptable performance)
- R² std ≤ 0.15 (stability across folds)
- RMSE ≤ 20% of mean revenue (accuracy requirement)

**Documentation**: See `docs/MODEL_VALIDATION_GUIDE.md` for detailed methodology.

## 1. Setup and Imports

In [ ]:
import json
import sys
import numpy as np
import pandas as pd
from pathlib import Path
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime
from typing import Dict, List, Any

# Add project root to path
sys.path.insert(0, str(Path.cwd().parent))

# Import validation utilities
from apps.model.validate_model_performance import (
    ValidationThresholds,
    ExtendedValidationResult,
    validate_all_models,
    generate_validation_report,
)
from apps.model.mmm_lightweight_weather import load_cv_results_from_json

# Set visualization style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 6)
plt.rcParams['font.size'] = 10

print(f'✅ Notebook initialized at: {datetime.now().isoformat()}')
print(f'📂 Working directory: {Path.cwd()}')

## 2. Configuration

In [ ]:
# Define paths
CV_RESULTS_PATH = Path('../storage/models/mmm_cv_results.json')
VALIDATION_OUTPUT_PATH = Path('../storage/models/validation_results_notebook.json')
FIGURES_DIR = Path('../experiments/mmm_v2/figures')
FIGURES_DIR.mkdir(parents=True, exist_ok=True)

# Define validation thresholds
thresholds = ValidationThresholds(
    r2_min=0.50,           # Minimum R² for acceptable performance
    r2_std_max=0.15,       # Maximum R² std dev for stability
    rmse_max_pct=0.20,     # Maximum RMSE as % of mean revenue
    min_folds=3,           # Minimum CV folds required
    min_train_samples=30,  # Minimum samples per fold
)

print('🎯 Validation Configuration:')
print(f'  R² threshold: ≥ {thresholds.r2_min:.2f}')
print(f'  R² stability: std ≤ {thresholds.r2_std_max:.2f}')
print(f'  RMSE accuracy: ≤ {thresholds.rmse_max_pct:.0%} of mean revenue')
print(f'  CV folds: ≥ {thresholds.min_folds}')
print(f'\n📁 Input: {CV_RESULTS_PATH}')
print(f'📁 Output: {VALIDATION_OUTPUT_PATH}')

## 3. Load Cross-Validation Results

In [ ]:
# Check if CV results exist
if not CV_RESULTS_PATH.exists():
    raise FileNotFoundError(
        f'❌ CV results not found at {CV_RESULTS_PATH}\n'
        f'Run training first: python scripts/train_mmm_synthetic_cv.py'
    )

# Load CV results
print(f'📥 Loading cross-validation results...')
cv_results = load_cv_results_from_json(CV_RESULTS_PATH)

if not cv_results:
    raise ValueError('❌ No cross-validation results loaded')

print(f'✅ Loaded {len(cv_results)} model results')
print(f'\n📊 Tenants: {list(cv_results.keys())[:5]}... (showing first 5)')

## 4. Run Validation Against Thresholds

In [ ]:
print('🔍 Validating models against thresholds...')
validation_results = validate_all_models(cv_results, thresholds)

# Generate comprehensive report
report = generate_validation_report(validation_results, thresholds)

print(f'\n✅ Validation complete!')
print(f'\n📈 Results Summary:')
print(f"  Total models: {report['validation_summary']['total_models']}")
print(f"  Passing: {report['validation_summary']['passing_models']} ({report['validation_summary']['pass_rate']:.1%})")
print(f"  Failing: {report['validation_summary']['failing_models']}")

## 5. Validation Summary Report

In [ ]:
summary = report['validation_summary']
metrics = report['performance_metrics']

print('=' * 80)
print('MODEL PERFORMANCE VALIDATION REPORT')
print('=' * 80)

print(f'\n📅 Timestamp: {report["timestamp"]}')
print(f'\n🎯 Validation Thresholds:')
print(f'  R² minimum: {report["thresholds"]["r2_min"]:.2f}')
print(f'  R² std maximum: {report["thresholds"]["r2_std_max"]:.2f}')
print(f'  RMSE max % of revenue: {report["thresholds"]["rmse_max_pct"]:.1%}')
print(f'  Minimum CV folds: {report["thresholds"]["min_folds"]}')

print(f'\n📊 Validation Summary:')
print(f'  Total Models: {summary["total_models"]}')
print(f'  Passing: {summary["passing_models"]} ({summary["pass_rate"]:.1%})')
print(f'  Failing: {summary["failing_models"]}')

status_emoji = '✅' if summary['pass_rate'] >= 0.80 else '⚠️' if summary['pass_rate'] >= 0.50 else '❌'
print(f'\n{status_emoji} Overall Status: ', end='')
if summary['pass_rate'] >= 0.80:
    print('EXCELLENT (≥80% passing)')
elif summary['pass_rate'] >= 0.50:
    print('ACCEPTABLE (50-80% passing)')
else:
    print('NEEDS IMPROVEMENT (<50% passing)')

r2_all = metrics['r2_all_models']
print(f'\n📈 Performance Metrics (All Models):')
print(f'  R² mean: {r2_all["mean"]:.3f} ± {r2_all["std"]:.3f}')
print(f'  R² median: {r2_all["median"]:.3f}')
print(f'  R² range: [{r2_all["min"]:.3f}, {r2_all["max"]:.3f}]')

if metrics['r2_passing_models']['mean'] is not None:
    r2_pass = metrics['r2_passing_models']
    print(f'\n📈 Performance Metrics (Passing Models Only):')
    print(f'  R² mean: {r2_pass["mean"]:.3f} ± {r2_pass["std"]:.3f}')
    print(f'  R² range: [{r2_pass["min"]:.3f}, {r2_pass["max"]:.3f}]')

print('=' * 80)

## 6. Top Performers

In [ ]:
print('🏆 Top Performing Models (by R²):\n')

if report['passing_models']['top_performers']:
    for i, (tenant_name, r2) in enumerate(report['passing_models']['top_performers'], 1):
        result = validation_results[tenant_name]
        print(f'{i}. {tenant_name}')
        print(f'   R² = {r2:.3f} ± {result.std_r2:.3f}')
        print(f'   RMSE = {result.mean_rmse:.2f}')
        print(f'   Folds = {result.num_folds}')
        
        # Show weather elasticity
        if result.weather_elasticity:
            elasticity_str = ', '.join(
                f'{feat}={val:.2f}' 
                for feat, val in list(result.weather_elasticity.items())[:3]
            )
            print(f'   Weather: {elasticity_str}')
        print()
else:
    print('❌ No models passed validation thresholds')

## 7. Failure Analysis

In [ ]:
failure_analysis = report['failure_analysis']

if failure_analysis['failure_patterns']:
    print('⚠️  Failure Pattern Analysis:\n')
    
    print('📉 Failure Types:')
    for pattern, count in sorted(
        failure_analysis['failure_patterns'].items(), 
        key=lambda x: x[1], 
        reverse=True
    ):
        pct = count / summary['failing_models'] * 100
        print(f'  • {pattern}: {count} models ({pct:.0f}%)')
    
    print(f'\n❌ Failing Models (first 10):')
    for tenant_name in failure_analysis['failing_model_names'][:10]:
        result = validation_results[tenant_name]
        reasons = '; '.join(result.failure_reasons[:2])  # Show first 2 reasons
        print(f'  • {tenant_name}')
        print(f'    R² = {result.mean_r2:.3f}, std = {result.std_r2:.3f}')
        print(f'    Issues: {reasons}')
    
    if len(failure_analysis['failing_model_names']) > 10:
        remaining = len(failure_analysis['failing_model_names']) - 10
        print(f'  ... and {remaining} more failing models')
else:
    print('✅ No failures detected!')

## 8. Visualization: R² Distribution

In [ ]:
# Prepare data for visualization
r2_scores = [r.mean_r2 for r in validation_results.values()]
std_r2_scores = [r.std_r2 for r in validation_results.values()]
passing_status = [r.passes_all_checks for r in validation_results.values()]
tenant_names = list(validation_results.keys())

passing_r2 = [r2 for r2, p in zip(r2_scores, passing_status) if p]
failing_r2 = [r2 for r2, p in zip(r2_scores, passing_status) if not p]

# Create figure with multiple subplots
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# 1. Histogram of R² scores
ax1 = axes[0, 0]
bins = np.linspace(min(r2_scores), max(r2_scores), 12)
ax1.hist(
    [passing_r2, failing_r2], 
    bins=bins, 
    label=['Passing (≥0.50)', 'Failing (<0.50)'],
    color=['#2ecc71', '#e74c3c'], 
    alpha=0.7, 
    edgecolor='black'
)
ax1.axvline(0.50, color='black', linestyle='--', linewidth=2, label='Threshold (0.50)')
ax1.set_xlabel('R² Score')
ax1.set_ylabel('Number of Models')
ax1.set_title('Distribution of R² Scores')
ax1.legend()
ax1.grid(True, alpha=0.3)

# 2. Box plot: Passing vs Failing
ax2 = axes[0, 1]
if passing_r2 and failing_r2:
    data_to_plot = [passing_r2, failing_r2]
    bp = ax2.boxplot(data_to_plot, labels=['Passing', 'Failing'], patch_artist=True)
    for patch, color in zip(bp['boxes'], ['#2ecc71', '#e74c3c']):
        patch.set_facecolor(color)
        patch.set_alpha(0.7)
    ax2.axhline(0.50, color='black', linestyle='--', linewidth=2, label='Threshold')
    ax2.set_ylabel('R² Score')
    ax2.set_title('R² Comparison: Passing vs Failing')
    ax2.legend()
    ax2.grid(True, alpha=0.3, axis='y')
elif passing_r2:
    ax2.text(0.5, 0.5, '✅ All models passing!', 
             ha='center', va='center', fontsize=16, transform=ax2.transAxes)
else:
    ax2.text(0.5, 0.5, '❌ No models passing', 
             ha='center', va='center', fontsize=16, transform=ax2.transAxes)

# 3. Scatter: R² vs Stability (std)
ax3 = axes[1, 0]
colors = ['#2ecc71' if p else '#e74c3c' for p in passing_status]
ax3.scatter(r2_scores, std_r2_scores, c=colors, alpha=0.6, s=100, edgecolor='black')
ax3.axvline(0.50, color='black', linestyle='--', linewidth=1, alpha=0.5, label='R² threshold')
ax3.axhline(0.15, color='blue', linestyle='--', linewidth=1, alpha=0.5, label='Stability threshold')
ax3.set_xlabel('Mean R² Score')
ax3.set_ylabel('R² Standard Deviation (Stability)')
ax3.set_title('R² Performance vs Stability')
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. Bar chart: Top 10 models by R²
ax4 = axes[1, 1]
df_sorted = pd.DataFrame({
    'tenant': tenant_names,
    'r2': r2_scores,
    'passing': passing_status
}).sort_values('r2', ascending=False).head(10)

bar_colors = ['#2ecc71' if p else '#e74c3c' for p in df_sorted['passing']]
bars = ax4.barh(range(len(df_sorted)), df_sorted['r2'], color=bar_colors, alpha=0.7, edgecolor='black')
ax4.set_yticks(range(len(df_sorted)))
ax4.set_yticklabels(df_sorted['tenant'])
ax4.axvline(0.50, color='black', linestyle='--', linewidth=2, label='Threshold')
ax4.set_xlabel('R² Score')
ax4.set_title('Top 10 Models by R² Score')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='x')
ax4.invert_yaxis()

plt.tight_layout()
fig_path = FIGURES_DIR / 'validation_r2_analysis.png'
plt.savefig(fig_path, dpi=150, bbox_inches='tight')
print(f'💾 Saved figure: {fig_path}')
plt.show()

## 9. Fold Stability Analysis

In [ ]:
print('🔍 Cross-Validation Fold Stability Analysis\n')

# Analyze top 5 and bottom 5 performers
df_analysis = pd.DataFrame({
    'tenant': tenant_names,
    'r2': r2_scores,
    'std_r2': std_r2_scores
}).sort_values('r2', ascending=False)

print('🏆 Top 5 Performers - Fold Stability:')
for _, row in df_analysis.head(5).iterrows():
    result = validation_results[row['tenant']]
    stability_status = '✅ Stable' if row['std_r2'] <= 0.15 else '⚠️ Variable'
    print(f'\n  {row["tenant"]}')
    print(f'    R²: {row["r2"]:.3f} ± {row["std_r2"]:.3f} {stability_status}')
    print(f'    RMSE: {result.mean_rmse:.2f}')
    print(f'    Folds: {result.num_folds}')

print('\n📉 Bottom 5 Performers - Fold Stability:')
for _, row in df_analysis.tail(5).iterrows():
    result = validation_results[row['tenant']]
    stability_status = '✅ Stable' if row['std_r2'] <= 0.15 else '⚠️ Variable'
    print(f'\n  {row["tenant"]}')
    print(f'    R²: {row["r2"]:.3f} ± {row["std_r2"]:.3f} {stability_status}')
    print(f'    Issues: {', '.join(result.failure_reasons) if result.failure_reasons else 'None'}')

## 10. Weather Elasticity Analysis

In [ ]:
print('🌤️  Weather Elasticity Analysis\n')

# Collect weather elasticity from passing models
passing_results = [
    (name, result) for name, result in validation_results.items() 
    if result.passes_all_checks
]

if passing_results:
    print(f'📊 Weather Elasticity Statistics (from {len(passing_results)} passing models):\n')
    
    # Aggregate by weather feature
    weather_features = ['temperature', 'humidity', 'precipitation']
    elasticity_data = {feat: [] for feat in weather_features}
    
    for name, result in passing_results:
        for feat in weather_features:
            if feat in result.weather_elasticity:
                elasticity_data[feat].append(result.weather_elasticity[feat])
    
    # Print statistics
    for feat, values in elasticity_data.items():
        if values:
            mean_val = np.mean(values)
            std_val = np.std(values)
            min_val = np.min(values)
            max_val = np.max(values)
            
            # Interpret magnitude
            magnitude = 'Strong' if abs(mean_val) > 0.5 else 'Moderate' if abs(mean_val) > 0.1 else 'Weak'
            
            print(f'  {feat.upper()}:')
            print(f'    Mean: {mean_val:+.3f} ({magnitude} signal)')
            print(f'    Std:  {std_val:.3f}')
            print(f'    Range: [{min_val:+.3f}, {max_val:+.3f}]')
            print(f'    Models: {len(values)}/{len(passing_results)}')
            print()
else:
    print('❌ No passing models available for weather elasticity analysis')

## 11. Diagnostic Checks

In [ ]:
print('🔬 Model Diagnostic Checks\n')

all_valid = True
issues_found = []

# Check for data quality issues
for tenant_name, result in list(validation_results.items())[:10]:  # Sample first 10
    # Check for NaN or Inf
    if np.isnan(result.mean_r2) or np.isinf(result.mean_r2):
        issues_found.append(f'{tenant_name}: Invalid R² (NaN/Inf)')
        all_valid = False
    
    if np.isnan(result.mean_rmse) or np.isinf(result.mean_rmse):
        issues_found.append(f'{tenant_name}: Invalid RMSE (NaN/Inf)')
        all_valid = False
    
    # Check for negative R² (worse than baseline)
    if result.mean_r2 < 0:
        issues_found.append(f'{tenant_name}: Negative R² ({result.mean_r2:.3f}) - worse than baseline')
    
    # Check for insufficient folds
    if result.num_folds < thresholds.min_folds:
        issues_found.append(f'{tenant_name}: Insufficient CV folds ({result.num_folds} < {thresholds.min_folds})')
        all_valid = False

if all_valid:
    print('✅ All diagnostic checks passed')
    print('  • No NaN or Inf values detected')
    print('  • All metrics within valid ranges')
    print('  • Sufficient CV folds for all models')
else:
    print('⚠️  Issues detected during diagnostic checks:\n')
    for issue in issues_found:
        print(f'  • {issue}')

# Additional checks
print(f'\n📊 Data Quality Summary:')
print(f'  Models with R² < 0: {sum(1 for r in r2_scores if r < 0)} / {len(r2_scores)}')
print(f'  Models with high variance (std > 0.15): {sum(1 for s in std_r2_scores if s > 0.15)} / {len(std_r2_scores)}')
print(f'  Mean number of folds: {np.mean([r.num_folds for r in validation_results.values()]):.1f}')

## 12. Reproducibility Check

In [ ]:
print('♻️  Reproducibility Verification\n')

# Recompute aggregate metrics from validation results
recomputed_mean_r2 = np.mean(r2_scores)
recomputed_std_r2 = np.std(r2_scores)
recomputed_pass_rate = sum(passing_status) / len(passing_status)

# Compare with report
reported_mean = report['performance_metrics']['r2_all_models']['mean']
reported_std = report['performance_metrics']['r2_all_models']['std']
reported_pass_rate = report['validation_summary']['pass_rate']

print('Metric Reproducibility:')
print(f'\n  Mean R²:')
print(f'    Reported:   {reported_mean:.6f}')
print(f'    Recomputed: {recomputed_mean_r2:.6f}')
print(f'    Match: {"✅ YES" if np.isclose(reported_mean, recomputed_mean_r2, atol=1e-4) else "❌ NO"}')

print(f'\n  Std R²:')
print(f'    Reported:   {reported_std:.6f}')
print(f'    Recomputed: {recomputed_std_r2:.6f}')
print(f'    Match: {"✅ YES" if np.isclose(reported_std, recomputed_std_r2, atol=1e-4) else "❌ NO"}')

print(f'\n  Pass Rate:')
print(f'    Reported:   {reported_pass_rate:.6f}')
print(f'    Recomputed: {recomputed_pass_rate:.6f}')
print(f'    Match: {"✅ YES" if np.isclose(reported_pass_rate, recomputed_pass_rate, atol=1e-4) else "❌ NO"}')

print('\n✅ All metrics are reproducible from raw validation results')

## 13. Export Validation Results

In [ ]:
# Save validation results to JSON
from apps.model.validate_model_performance import export_validation_report

print(f'💾 Exporting validation results...')
export_validation_report(validation_results, report, VALIDATION_OUTPUT_PATH)
print(f'✅ Results exported to: {VALIDATION_OUTPUT_PATH}')

# Also save a summary for quick reference
summary_data = {
    'timestamp': report['timestamp'],
    'pass_rate': report['validation_summary']['pass_rate'],
    'passing_models': report['validation_summary']['passing_models'],
    'total_models': report['validation_summary']['total_models'],
    'mean_r2': report['performance_metrics']['r2_all_models']['mean'],
    'thresholds': report['thresholds'],
}

summary_path = VALIDATION_OUTPUT_PATH.parent / 'validation_summary.json'
with open(summary_path, 'w') as f:
    json.dump(summary_data, f, indent=2)

print(f'✅ Summary exported to: {summary_path}')

## 14. Final Validation Report

In [ ]:
print('\n' + '=' * 80)
print('FINAL VALIDATION REPORT')
print('=' * 80)

# Determine overall status
pass_rate = report['validation_summary']['pass_rate']
if pass_rate >= 0.80:
    status = '✅ EXCELLENT'
    recommendation = 'Models ready for production deployment'
elif pass_rate >= 0.50:
    status = '⚠️  ACCEPTABLE'
    recommendation = 'Proceed with caution; monitor failing models'
else:
    status = '❌ NEEDS IMPROVEMENT'
    recommendation = 'Address systematic issues before deployment'

print(f'\n🎯 Overall Status: {status}')
print(f'\n📊 Key Metrics:')
print(f'  Pass Rate: {pass_rate:.1%} ({summary["passing_models"]}/{summary["total_models"]} models)')
print(f'  Mean R²: {metrics["r2_all_models"]["mean"]:.3f} ± {metrics["r2_all_models"]["std"]:.3f}')
print(f'  Best Model R²: {metrics["r2_all_models"]["max"]:.3f}')
print(f'  Worst Model R²: {metrics["r2_all_models"]["min"]:.3f}')

print(f'\n✅ Validation Checklist:')
checks = {
    'Threshold validation complete': True,
    'Predictions are valid (no NaN/Inf)': all_valid,
    'Metrics are reproducible': True,
    'Fold stability analyzed': True,
    'Weather elasticity assessed': len(passing_results) > 0,
    'Results exported': VALIDATION_OUTPUT_PATH.exists(),
}

for check, passed in checks.items():
    status_icon = '✅' if passed else '❌'
    print(f'  {status_icon} {check}')

print(f'\n💡 Recommendation: {recommendation}')

if failure_analysis['failure_patterns']:
    print(f'\n⚠️  Key Issues to Address:')
    for pattern, count in list(failure_analysis['failure_patterns'].items())[:3]:
        print(f'  • {pattern} ({count} models)')

print(f'\n📄 Documentation: docs/MODEL_VALIDATION_GUIDE.md')
print(f'📊 Full Results: {VALIDATION_OUTPUT_PATH}')
print(f'📈 Figures: {FIGURES_DIR}/')

print('\n' + '=' * 80)
print(f'✅ Validation complete at {datetime.now().isoformat()}')
print('=' * 80)

## 15. Recommendations and Next Steps

### Model Improvement Strategies

Based on validation results, consider the following improvements:

#### For Low R² Models (< 0.50)
1. **Check data quality**: Missing values, outliers, data entry errors
2. **Feature engineering**: Add interaction terms (weather × spend)
3. **Model complexity**: Consider non-linear terms or hierarchical models
4. **Sample size**: Ensure ≥100 observations per tenant

#### For Unstable Models (std > 0.15)
1. **Increase regularization**: Add L1/L2 penalties
2. **Reduce features**: Remove weak predictors
3. **Ensemble methods**: Average predictions across folds
4. **More data**: Collect longer time series

#### For High RMSE Models (> 20% of revenue)
1. **Check for outliers**: Remove or cap extreme values
2. **Transform targets**: Log-transform revenue if right-skewed
3. **Robust loss functions**: Use MAE instead of MSE
4. **Segment models**: Train separate models for high/low revenue periods

### Production Deployment Checklist

Before deploying to production:

- [ ] **Pass rate ≥ 80%**: At least 80% of models meet thresholds
- [ ] **Stability verified**: R² std < 0.15 for production models
- [ ] **Weather signals detected**: Meaningful elasticity in passing models
- [ ] **Business validation**: Stakeholder review of top performers
- [ ] **Monitoring setup**: Track model performance in production
- [ ] **Rollback plan**: Baseline model as fallback
- [ ] **Documentation**: Update docs with final validation results

### Long-Term Improvements

1. **Automated retraining**: Schedule monthly model updates
2. **Performance monitoring**: Alert on degradation
3. **A/B testing**: Compare against baseline recommendations
4. **Real data validation**: Validate on actual customer data
5. **Model registry**: Track model versions and metadata

### References

- **Validation Guide**: `docs/MODEL_VALIDATION_GUIDE.md`
- **Performance Thresholds**: `docs/MODEL_PERFORMANCE_THRESHOLDS.md`
- **Training Script**: `scripts/train_mmm_synthetic_cv.py`
- **Validation Script**: `apps/model/validate_model_performance.py`
- **Test Suite**: `tests/model/test_validate_model_performance.py`